In [4]:
#!pip install openai==0.28
#!pip install python-dotenv

In [61]:
import openai
import os
import pandas as pd
import time
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from collections import defaultdict
import json
from langchain_text_splitters import RecursiveJsonSplitter

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [62]:
embedding_function = OpenAIEmbeddings()

# Create the language model
llm = ChatOpenAI(model='gpt-4o', temperature=0.0)

analysis_prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    """
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{context}"
     """,
     ),
    ("human", "{question}")
    ]
)

In [63]:
def csv_files_to_json(file_paths, encoding='cp949'):
    # Initialize a default dictionary to store dictionaries for each ID
    json_data = defaultdict(lambda: defaultdict(list))
    
    # Iterate through each CSV file
    for file_path in file_paths:
        # Extract the file name without extension to use as a key
        file_key = file_path.split('/')[-1].split('.')[0]
        
        # Read the CSV file into a DataFrame with cp949 encoding
        df = pd.read_csv(file_path, encoding=encoding)
        
        # Group the data by 'id'
        for id, group in df.groupby('id'):
            # Convert the group data to a dictionary without the 'id' column
            record = group.drop(columns=['id']).to_dict(orient='records')
            # Append the record under the corresponding file name for the given id
            json_data[str(id)][file_key].extend(record)
    
    # Convert the default dictionary to a regular dictionary and then to a JSON string
    json_str = json.dumps(dict(json_data), indent=4)
    
    return json_str

In [50]:
# # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
# file_paths = ['./data/inbody.csv', './data/patients.csv', './data/vital.csv']
# 
# inbody_json = csv_files_to_json(file_paths)
# json_data = json.loads(inbody_json)
# 
# # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
# target_id = '1'
# 
# # 해당 id가 존재하는지 확인
# if target_id in json_data:
#     id_data = json_data[target_id]
#     print(f"Data for ID {target_id}:")
#     json_output = json.dumps(id_data)
# else:
#     print(f"ID {target_id} not found.")
# 
# json_data = json.loads(json_output)
# 
# splitter = RecursiveJsonSplitter(max_chunk_size=300)
# 
# 
# # The splitter can also output documents
# docs = splitter.create_documents(texts=[json_data])
# 
# # or a list of strings
# texts = splitter.split_text(json_data=json_data)
# print(texts[0])
# print(texts[1])

Data for ID 1:
{"inbody": [{"date": "2024-06-12", "time": "0:07:42", "body_fat_mass": 15.2, "total_body_water": 28.3, "fat_free_mass": 38.6, "skeletal_muscle_mass": 20.8, "soft_lean_mass": 36.4, "bmi": 21.1, "percent_body_fat": 28.2, "waist_hip_ratio": 0.86, "bmc": 2.15, "bcm": 24.9, "bmr": 1203, "visceral_fat_level": 6}, {"date": "2024-07-03", "time": "23:42:07", "body_fat_mass": 17.0, "total_body_water": 26.7, "fat_free_mass": 36.4, "skeletal_muscle_mass": 19.5, "soft_lean_mass": 34.3, "bmi": 22.3, "percent_body_fat": 31.8, "waist_hip_ratio": 0.87, "bmc": 2.06, "bcm": 23.5, "bmr": 1156, "visceral_fat_level": 7}, {"date": "2024-07-17", "time": "8:55:22", "body_fat_mass": 16.8, "total_body_water": 26.5, "fat_free_mass": 36.1, "skeletal_muscle_mass": 19.3, "soft_lean_mass": 34.1, "bmi": 22.1, "percent_body_fat": 31.8, "waist_hip_ratio": 0.86, "bmc": 2.05, "bcm": 23.2, "bmr": 1149, "visceral_fat_level": 7}]}
{"patients": [{"pat_sex": "F", "pat_birth": 1960}], "vital": [{"time": "2024-05-

In [64]:
def inbody_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/inbody.csv']
    
    inbody_json = csv_files_to_json(file_paths)
    json_data = json.loads(inbody_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 인바디 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
        
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    # texts = splitter.split_text(json_data=json_data)

    #벡터스토어 생성
    inbody_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    inbody_retriever = inbody_vectorstore.as_retriever()
    
    inbody_chain = (
        {"context": inbody_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    inbody_response = inbody_chain.invoke(question)
    return inbody_response

In [65]:
def vital_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/vital.csv']
    
    vital_json = csv_files_to_json(file_paths)
    json_data = json.loads(vital_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 바이탈 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    vital_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    vital_retriever = vital_vectorstore.as_retriever()
    
    vital_chain = (
        {"context": vital_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    vital_response = vital_chain.invoke(question)
    return vital_response

In [66]:
def patients_analysis(embedding_function, llm, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/patients.csv']
    
    patients_json = csv_files_to_json(file_paths)
    json_data = json.loads(patients_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 환자 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    patients_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    patients_retriever = patients_vectorstore.as_retriever()
    
    patients_query = f"""
    context에 있는 정보를 정리하여 간단하게 출력하라. 현재 날짜는 :{time.strftime('%Y.%m.%d')}이며 context의 pat_sex는 환자의 성별, pat_birth는 환자의 탄생연도를 뜻한다.\n\n
    """
    
    patients_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    patients_query,
     ),
    ("human", "{question}")
    ]
)
    
    patients_chain = (
        {"context": patients_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    patients_response = patients_chain.invoke(question)
    return patients_response

In [67]:
def final_response(id, question):
    inbody_response = inbody_analysis(embedding_function, llm, analysis_prompt, id, question)
    vital_response = vital_analysis(embedding_function, llm, analysis_prompt, id, question)
    patients_response = patients_analysis(embedding_function, llm, id, question)
    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    """
    
    final_query = f"""
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{total_analysis}"""
    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
         final_query         
         ),
        ("human", "{question}")
    ])
    final_chain = (
        {"question": RunnablePassthrough()}
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(question)
    return response

In [68]:
print(final_response('0','건강정보에 대한 종합적인 소견을 말해'))

0에 대한 인바디 정보 분석 중...
0에 대한 바이탈 정보 분석 중...
0에 대한 환자 정보 분석 중...
종합 분석:
2024년 6월 30일과 2024년 7월 17일의 두 차례에 걸친 인바디 검사 결과를 비교해보면, 전반적으로 체성분의 변화가 미미하지만 긍정적인 방향으로 진행되고 있음을 알 수 있습니다. 체지방량은 18.2kg에서 17.5kg으로 감소하였고, 체지방률도 27.7%에서 27.1%로 감소하였습니다. 이는 체지방이 줄어들고 있음을 의미합니다. 또한, 체질량지수(BMI)도 22.7에서 22.4로 약간 감소하였습니다. 골격근량은 25.9kg에서 25.7kg으로 약간 감소하였으나, 이는 큰 변화는 아닙니다. 내장지방수치는 두 번 모두 7로 동일하게 유지되고 있습니다.

환자의 혈압, 체중, 혈당, 맥박, 체온 등의 주요 건강 지표를 분석한 결과, 전반적으로 안정적인 상태를 유지하고 있는 것으로 보입니다. 혈압은 수축기 혈압(SBP)과 이완기 혈압(DBP) 모두 정상 범위 내에 있으며, 큰 변동 없이 일정하게 유지되고 있습니다. 체중은 약간의 변동이 있지만 큰 변화는 없으며, 이는 일상적인 체중 변동으로 볼 수 있습니다. 혈당 수치는 대부분 정상 범위에 속하지만, 2024년 6월 30일에 측정된 혈당 수치가 121로 약간 높게 나타났습니다. 맥박과 체온 역시 정상 범위 내에 있으며, 산소포화도(SPO2)도 양호한 상태를 유지하고 있습니다.

종합적인 조언:
현재 체성분의 변화는 긍정적인 방향으로 진행되고 있습니다. 체지방량과 체지방률이 감소하고 있는 것은 좋은 신호입니다. 하지만 골격근량이 약간 감소한 점은 주의가 필요합니다. 근육량을 유지하거나 증가시키기 위해서는 규칙적인 근력 운동과 충분한 단백질 섭취가 중요합니다. 내장지방수치가 7로 유지되고 있는 점도 주목할 필요가 있습니다. 내장지방을 줄이기 위해서는 지속적인 유산소 운동과 건강한 식습관이 필요합니다. 전반적으로 현재의 건강 상태를 유지하면서, 근육량을 증가시키고 내장지방을 줄이는 방향으로 생

In [69]:
print(final_response('0','추천하는 운동을 알려줘'))

0에 대한 인바디 정보 분석 중...
0에 대한 바이탈 정보 분석 중...
0에 대한 환자 정보 분석 중...
종합 분석:
환자의 체성분 분석 결과를 보면, 체지방량과 체지방률이 약간 높은 편입니다. 특히, 내장지방 수준이 7로 나타나고 있어, 내장지방을 줄이는 것이 중요합니다. 근육량은 적절한 수준을 유지하고 있지만, 체지방률을 낮추기 위해 유산소 운동과 근력 운동을 병행하는 것이 좋습니다.

환자의 혈압, 체온, 체중, 혈당 수치, 맥박 등 여러 가지 건강 지표를 종합적으로 분석한 결과, 전반적으로 건강 상태는 양호한 편입니다. 혈압은 정상 범위 내에 있으며, 체중도 큰 변동 없이 안정적으로 유지되고 있습니다. 혈당 수치는 다소 변동이 있지만, 대체로 정상 범위에 가깝습니다. 맥박과 산소포화도 역시 정상 범위에 있습니다.

종합적인 조언:
1. 유산소 운동: 체지방을 줄이기 위해 주 3-5회, 30-60분 정도의 유산소 운동을 추천합니다. 걷기, 조깅, 자전거 타기, 수영 등이 효과적입니다. 이는 심혈관 건강을 유지하고 혈압을 안정시키는 데 도움이 됩니다.

2. 근력 운동: 근육량을 유지하고 증가시키기 위해 주 2-3회, 전신 근력 운동을 추천합니다. 스쿼트, 런지, 푸시업, 덤벨 리프팅 등의 운동을 포함시켜 보세요. 근력 운동은 근육량을 유지하고 대사율을 높이는 데 도움이 됩니다.

3. 코어 운동: 허리와 복부의 근력을 강화하기 위해 플랭크, 브릿지 등의 코어 운동을 추가하는 것이 좋습니다.

4. 유연성 운동: 요가나 스트레칭 운동을 통해 유연성을 유지하고 근육의 긴장을 완화시킬 수 있습니다. 이는 부상 예방과 전반적인 신체 균형을 유지하는 데 도움이 됩니다.

5. 균형 운동: 균형 감각을 향상시키기 위해 태극권이나 특정 균형 운동을 포함시키는 것도 좋습니다. 이는 특히 나이가 들면서 발생할 수 있는 낙상 위험을 줄이는 데 도움이 됩니다.

6. 스트레칭: 운동 후에는 충분한 스트레칭을 통해 근육의 유연성을 유지하고 부상을 예방하세요.

7. 식이

In [70]:
print(final_response('0','추천하는 식단들을 설명해'))

0에 대한 인바디 정보 분석 중...
0에 대한 바이탈 정보 분석 중...
0에 대한 환자 정보 분석 중...
종합 분석:
환자의 혈압, 체온, 체중, 혈당 수치, 맥박 등 여러 가지 건강 지표를 종합적으로 분석한 결과, 전반적으로 건강 상태는 양호한 편입니다. 혈압은 정상 범위 내에 있으며, 체중도 큰 변동 없이 안정적으로 유지되고 있습니다. 혈당 수치는 다소 변동이 있지만, 대체로 정상 범위에 가깝습니다. 산소포화도와 체온도 정상 범위 내에 있습니다.

종합적인 조언:
1. **균형 잡힌 식단**: 다양한 영양소를 골고루 섭취할 수 있는 식단을 유지하는 것이 중요합니다. 채소, 과일, 단백질, 건강한 지방, 그리고 복합 탄수화물을 포함한 식단을 추천합니다.
   - **채소와 과일**: 브로콜리, 시금치, 당근, 토마토, 사과, 베리류 등 다양한 색상의 채소와 과일을 섭취하세요.
   - **단백질**: 닭고기, 생선, 두부, 콩류, 달걀 등 다양한 단백질원을 포함하세요.
   - **건강한 지방**: 아보카도, 올리브 오일, 견과류, 씨앗류 등 건강한 지방을 섭취하세요.
   - **복합 탄수화물**: 현미, 귀리, 퀴노아, 통밀빵 등 복합 탄수화물을 선택하세요.

2. **혈당 관리**: 혈당 수치가 다소 변동이 있으므로, 혈당 지수가 낮은 음식을 선택하는 것이 좋습니다. 정제된 탄수화물보다는 통곡물, 채소, 과일 등을 섭취하세요.
   - **낮은 혈당 지수 음식**: 귀리, 퀴노아, 보리, 콩류, 녹색 잎채소, 베리류 등을 포함하세요.
   - **피해야 할 음식**: 흰 빵, 흰 쌀, 설탕이 많이 들어간 음식은 피하세요.

3. **수분 섭취**: 충분한 수분 섭취는 전반적인 건강에 매우 중요합니다. 하루에 최소 8잔의 물을 마시는 것을 권장합니다.
   - **수분 섭취 방법**: 물, 허브티, 무가당 음료 등을 통해 수분을 보충하세요.

4. **규칙적인 운동**: 규칙적인 운동은 혈압과 혈당을 안정시키는 데 도움이 됩니다. 주 3-4회, 30